In [1]:
#Varibales for Code
data = [
("AMD", "COK", "2024-02-24 08:00:00", "2024-02-24 10:30:00", "2024-02-24 08:50:00", "2024-02-24 11:25:00", "Akasa Air", "airTraffic"),
("COK", "SXR", "2024-02-25 12:00:00", "2024-02-25 18:30:00", "2024-02-25 12:15:00", "2024-02-25 19:15:00", "Vistara", "mechanical"),
("COK", "AMD", "2024-03-24 08:00:00", "2024-03-24 10:40:00", "2024-03-24 08:17:00", "2024-03-24 11:00:00", "Akasa Air", "storm"),
("SXR", "AMD", "2024-04-28 05:30:00", "2024-04-28 07:35:00", "2024-04-28 05:50:00", "2024-04-28 07:55:00", "Vistara", "visibility"),
("AMD", "SXR", "2024-07-25 07:00:00", "2024-07-25 09:10:00", "2024-07-25 07:10:00", "2024-07-25 09:20:00", "Akasa Air", "airTraffic"),
("AMD", "CCU", "2024-03-28 18:00:00", "2024-03-28 20:25:00", "2024-03-28 19:00:00", "2024-03-28 21:15:00", "Vistara", "storm"),
("SXR", "COK", "2024-06-28 23:00:00", "2024-06-29 04:40:00", "2024-06-28 23:40:00", "2024-06-29 05:20:00", "Vistara", "airTraffic"),
("COK", "CCU", "2024-06-24 18:00:00", "2024-06-24 21:05:00", "2024-06-24 18:05:00", "2024-06-24 21:25:00", "IndiGo", "visibility"),
("CCU", "AMD", "2024-07-25 04:00:00", "2024-07-25 06:45:00", "2024-07-25 04:30:00", "2024-07-25 07:05:00", "IndiGo", "airTraffic"),
("CCU", "COK", "2024-02-28 14:00:00", "2024-02-28 17:10:00", "2024-02-28 14:10:00", "2024-02-28 17:40:00", "IndiGo", "mechanical"),
("CCU", "SXR", "2024-04-21 17:00:00", "2024-04-21 19:50:00", "2024-04-21 17:30:00", "2024-04-21 20:05:00", "IndiGo", "airTraffic")
]

In [2]:
#Schema to be imposed
schm = ("origin_airport", "destination_airport", "scheduled_departure_time", "scheduled_arrival_time", "actual_departure_time", "actual_arrival_time", "carrier", "delay_reason")

In [3]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=e53776193a1f5bfdb32d96fccd2a4b2bfb849eb06023be55e1af0afdc3fcfa0c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [5]:
#Create a SparkSession
s1 = SparkSession.builder.appName('pyspark').getOrCreate()

In [6]:
#Create dataframe for the given data using enforced Schema method
df = s1.createDataFrame(data,schema = schm)

In [7]:
df.show()

+--------------+-------------------+------------------------+----------------------+---------------------+-------------------+---------+------------+
|origin_airport|destination_airport|scheduled_departure_time|scheduled_arrival_time|actual_departure_time|actual_arrival_time|  carrier|delay_reason|
+--------------+-------------------+------------------------+----------------------+---------------------+-------------------+---------+------------+
|           AMD|                COK|     2024-02-24 08:00:00|   2024-02-24 10:30:00|  2024-02-24 08:50:00|2024-02-24 11:25:00|Akasa Air|  airTraffic|
|           COK|                SXR|     2024-02-25 12:00:00|   2024-02-25 18:30:00|  2024-02-25 12:15:00|2024-02-25 19:15:00|  Vistara|  mechanical|
|           COK|                AMD|     2024-03-24 08:00:00|   2024-03-24 10:40:00|  2024-03-24 08:17:00|2024-03-24 11:00:00|Akasa Air|       storm|
|           SXR|                AMD|     2024-04-28 05:30:00|   2024-04-28 07:35:00|  2024-04-28 05:

In [8]:
#Filter the dataframe to display all delayed flights
df1 = df.withColumn('delay_departure',(unix_timestamp('actual_departure_time') - unix_timestamp('scheduled_departure_time')) / 60)
df1.where(col('delay_departure') > 0).show()

+--------------+-------------------+------------------------+----------------------+---------------------+-------------------+---------+------------+---------------+
|origin_airport|destination_airport|scheduled_departure_time|scheduled_arrival_time|actual_departure_time|actual_arrival_time|  carrier|delay_reason|delay_departure|
+--------------+-------------------+------------------------+----------------------+---------------------+-------------------+---------+------------+---------------+
|           AMD|                COK|     2024-02-24 08:00:00|   2024-02-24 10:30:00|  2024-02-24 08:50:00|2024-02-24 11:25:00|Akasa Air|  airTraffic|           50.0|
|           COK|                SXR|     2024-02-25 12:00:00|   2024-02-25 18:30:00|  2024-02-25 12:15:00|2024-02-25 19:15:00|  Vistara|  mechanical|           15.0|
|           COK|                AMD|     2024-03-24 08:00:00|   2024-03-24 10:40:00|  2024-03-24 08:17:00|2024-03-24 11:00:00|Akasa Air|       storm|           17.0|
|   

In [9]:
#Calculate Average delay per source airport
df1.groupBy('origin_airport').agg({'delay_departure':'avg'}).show()

+--------------+--------------------+
|origin_airport|avg(delay_departure)|
+--------------+--------------------+
|           SXR|                30.0|
|           AMD|                40.0|
|           COK|  12.333333333333334|
|           CCU|  23.333333333333332|
+--------------+--------------------+



In [10]:
##Calculate Average delay per destination airport
df1.groupBy('destination_airport').agg({'delay_departure':'avg'}).show()

+-------------------+--------------------+
|destination_airport|avg(delay_departure)|
+-------------------+--------------------+
|                SXR|  18.333333333333332|
|                AMD|  22.333333333333332|
|                COK|  33.333333333333336|
|                CCU|                32.5|
+-------------------+--------------------+



In [11]:
#Calculate Average delay per airline
df1.groupBy('carrier').agg({'delay_departure':'avg'}).show()

+---------+--------------------+
|  carrier|avg(delay_departure)|
+---------+--------------------+
|Akasa Air|  25.666666666666668|
|  Vistara|               33.75|
|   IndiGo|               18.75|
+---------+--------------------+



In [12]:
#Fetch airline with maximum delay (in single flight)
df1.groupBy('carrier').agg({'delay_departure':'max'}).orderBy(max('delay_departure').desc()).limit(1).show()

+-------+--------------------+
|carrier|max(delay_departure)|
+-------+--------------------+
|Vistara|                60.0|
+-------+--------------------+



In [13]:
#Fetch airline with maximum delay (overall time delay)
df1.groupBy('carrier').agg({'delay_departure':'sum'}).orderBy(sum('delay_departure').desc()).limit(1).show()

+-------+--------------------+
|carrier|sum(delay_departure)|
+-------+--------------------+
|Vistara|               135.0|
+-------+--------------------+



In [14]:
#Fetch airline with minimum delay (in single flight)
df1.groupBy('carrier').agg({'delay_departure':'min'}).orderBy(min('delay_departure')).limit(1).show()

+-------+--------------------+
|carrier|min(delay_departure)|
+-------+--------------------+
| IndiGo|                 5.0|
+-------+--------------------+



In [15]:
#Fetch airline with minimum delay (overall time delay)
df1.groupBy('carrier').agg({'delay_departure':'sum'}).orderBy(sum('delay_departure')).limit(1).show()

+-------+--------------------+
|carrier|sum(delay_departure)|
+-------+--------------------+
| IndiGo|                75.0|
+-------+--------------------+



In [16]:
#Calculate Average delay per reason
df1.groupBy('delay_reason').agg({'delay_departure':'avg'}).show()

+------------+--------------------+
|delay_reason|avg(delay_departure)|
+------------+--------------------+
|  visibility|                12.5|
|  airTraffic|                32.0|
|       storm|                38.5|
|  mechanical|                12.5|
+------------+--------------------+



In [17]:
#Find most recurring cause of delays
df1.groupBy('delay_reason').agg({'delay_reason':'count'}).orderBy(count('delay_reason').desc()).limit(1).show()

+------------+-------------------+
|delay_reason|count(delay_reason)|
+------------+-------------------+
|  airTraffic|                  5|
+------------+-------------------+



In [24]:
#Identify the airline with best recovery time
# (Even though the flight started late, the airline recovered a few minutes in flight time)
df1 = df1.withColumn('delay_arrival',(unix_timestamp('actual_arrival_time') - unix_timestamp('scheduled_arrival_time')) / 60)
df1.where(col('delay_arrival') < col('delay_departure')).show()


+--------------+-------------------+------------------------+----------------------+---------------------+-------------------+-------+------------+---------------+-------------+
|origin_airport|destination_airport|scheduled_departure_time|scheduled_arrival_time|actual_departure_time|actual_arrival_time|carrier|delay_reason|delay_departure|delay_arrival|
+--------------+-------------------+------------------------+----------------------+---------------------+-------------------+-------+------------+---------------+-------------+
|           AMD|                CCU|     2024-03-28 18:00:00|   2024-03-28 20:25:00|  2024-03-28 19:00:00|2024-03-28 21:15:00|Vistara|       storm|           60.0|         50.0|
|           CCU|                AMD|     2024-07-25 04:00:00|   2024-07-25 06:45:00|  2024-07-25 04:30:00|2024-07-25 07:05:00| IndiGo|  airTraffic|           30.0|         20.0|
|           CCU|                SXR|     2024-04-21 17:00:00|   2024-04-21 19:50:00|  2024-04-21 17:30:00|2024